In [1]:
import math
import json
import regex as re
from itertools import permutations

# Get inputs

In [2]:
#with open("test_input.txt") as f:
with open("input.txt") as f:
    inp = f.readlines()
    inp = [line.rstrip() for line in inp]
cleaned = [json.loads(x) for x in inp]
cleaned[:5]

[[[3, [8, [2, 1]]], [[[0, 6], [0, 2]], 3]],
 [[[1, [8, 5]], [[3, 9], 0]], 2],
 [5, [[5, [3, 8]], [7, 4]]],
 [1, [[[0, 4], [8, 5]], 6]],
 [[[1, [0, 3]], 2], [2, [0, [7, 9]]]]]

## Q1

In [3]:
def get_magnitude(num):
    """3 * left elements + 2 * right elements"""
    
    left = num[0]
    right = num[1]
    
    if isinstance(left,int) and isinstance(right,int):
        return left*3 + right*2

    elif isinstance(left,list) and isinstance(right,list):
        left = get_magnitude(left)
        right = get_magnitude(right)

    elif isinstance(left,list):
        left = get_magnitude(left)
        
    else:
        right = get_magnitude(right)
    
    return get_magnitude([left, right])

get_magnitude([[[[6,6],[7,6]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[7,8],[9,9]]]])

4140

In [4]:
def next_digit(s):
    """Returns index of next digit or False"""
    for i, c in enumerate(s):
        if c.isdigit():
            return i
    return False

next_digit(']]]],[1,1]]')

6

In [5]:
def is_snail_num(s):
    """Matches snail number"""
    r = re.compile(r'\d+,\s*\d+')
    return True if re.match(r, s) else False

print(is_snail_num('92, 2]][[1,2]]'))

True


In [6]:
def extract_snail_num(s):
    """Returns snail numbers as list"""
    r = re.compile(r'(\d+,\s*\d+)')
    m = re.match(r,s).group(0)

    return [x.strip() for x in m.split(',')]
extract_snail_num('92, 2]]][')

['92', '2']

In [7]:
def get_digit(s):
    """Returns full digit"""
    r = re.compile(r'\d+')
    return re.match(r,s).group(0)

get_digit('92],2')

'92'

In [8]:
def explode(num):
    """Explodes a pair"""
    
    depth = 0
    s = str(num)
    s = s.replace(" ","")
    left_index = False
    
    for i,c in enumerate(s):
        
        if c == '[':
            depth += 1
            
        elif c == ']':
            depth -= 1
            
        elif c == ',':
            continue

        elif depth > 4 and is_snail_num(s[i:]):
            left, right = extract_snail_num(s[i:])
            # print(f"{i*'-'}^ Exploding [{left}, {right}]\n")

            if left_index:
                old_left = get_digit(s[left_index:])
                n = str(int(old_left) + int(left))
                
                s = s[:left_index] + n + s[left_index+len(old_left):]
                
                i = i - len(old_left) + len(n)

            right_index = next_digit(s[i+1+len(left)+len(right):])
            if right_index:
                right_index = i+1+len(left)+len(right)+right_index
                old_right = get_digit(s[right_index:])
                
                n = str(int(old_right) + int(right))
                s = s[:right_index] + n + s[right_index+len(old_right):]
            
            # replace exploded pair with zero
            replaced = s[:i-1] + '0' + s[i+len(left)+len(right)+2:]
            return json.loads(replaced)
        
        else:
            while c.isdigit():
                c = s[i-1]
                i -= 1
            left_index = i + 1
            
    return False

explode([[[[4,0],[5,4]],[[7,0],[15,5]]],[10,[[0,[11,3]],[[6,3],[8,8]]]]])

[[[[4, 0], [5, 4]], [[7, 0], [15, 5]]], [10, [[11, 0], [[9, 3], [8, 8]]]]]

In [9]:
def split(num):
    
    s = str(num)
    s = s.replace(" ","")
    for i, c in enumerate(s):
        if c.isdigit():
            c = get_digit(s[i:])
            
            if len(c) > 1:
                # print(f"{i*'-'}^ Splitting {c}\n")

                first = math.floor(int(c)/2)
                last = math.ceil(int(c)/2)
                s = s[:i] +'['+ str(first) +','+ str(last) +']'+ s[i+len(c):]
                
                return json.loads(s)
    
    return False

split([[[[4,0],[5,4]],[[7,7],[0,13]]],[15,[[0,17],[0,6]]]])

[[[[4, 0], [5, 4]], [[7, 7], [0, [6, 7]]]], [15, [[0, 17], [0, 6]]]]

In [10]:
def search_explode(num, depth = 0):
    for i in num:
        if isinstance(i,int):
            if depth > 3:
                return "explode"
            
        elif isinstance(i,list):
            action = search_explode(i, depth+1)
            if action:
                return action
            
    return False

In [11]:
def search_split(num):
    for i, x in enumerate(num):
        if isinstance(x,int):
            if x >= 10:
                return "split"
            
        elif isinstance(x,list):
            action = search_split(x)
            if action:
                return action
        
    return False

search_split([[[[4,0],[5,4]],[[7,7],[0,13]]],[15,[[0,17],[0,6]]]])

'split'

In [12]:
def action_required(num):
    """Loop through num, check if num needs to be exploded or split"""
    
    action = search_explode(num)
    if action:
        return action
    
    action = search_split(num)
    if action:
        return action
    
    return False

action_required([[[[4,0],[5,4]],[[7,0],[15,5]]],[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]])

'explode'

In [13]:
def snailfish(homework):
    
    current_num = homework[0]

    for lines in homework[1:]:
        # print(str(current_num).replace(" ",""), ' + ', str(lines).replace(" ",""))
        current_num = [current_num] + [lines]
        # print(f'Adding numbers\n{str(current_num).replace(" ","")}')
        
        action = action_required(current_num)
        while action:
            if action == "explode":
                current_num = explode(current_num)
            else:
                current_num = split(current_num)
            # print(str(current_num).replace(" ",""))
            
            action = action_required(current_num)

    return current_num

In [14]:
final = snailfish(cleaned)
print(final)
print(f"Part 1: {get_magnitude(final)}")

[[[[7, 6], [6, 7]], [[7, 7], [7, 8]]], [[[8, 0], [8, 7]], [[6, 8], [6, 0]]]]
Part 1: 3981


## Q2

In [15]:
magnitude = 0
for i in permutations(range(len(cleaned)),2):
    tmp = get_magnitude(snailfish([cleaned[i[0]],cleaned[i[1]]]))
    magnitude = max(magnitude, tmp)
magnitude

4687

In [16]:
print(f"Part 2: {magnitude}")

Part 2: 4687
